In [1]:
import random 
import warnings
import numpy as np 
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from utils.modelos import *
from utils.analisis import *
from utils.transformacion import *
from utils.recomendaciones import *

from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEANING = '../data/cleaning'


random.seed(VAR_SEED)
np.random.seed(VAR_SEED)
warnings.filterwarnings("ignore")


EJERCICIOS = pd.read_csv(f"{VAR_DIR_DATA_CLEANING}/ejercicios.csv", encoding="latin1")
ESTUDIANTES = pd.read_csv(f"{VAR_DIR_DATA_CLEANING}/estudiantes.csv", encoding="latin1")


# 'h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4'
# 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'puntos', 'enunciado', 'dificultad', 'score_a', 'score_d', 'score_p', 'score_s'
df_items = EJERCICIOS[['id_ejercicio','h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4']] 


# estudiantes_reprobados = ESTUDIANTES.query(" `solemne_1` < 4.0 and `solemne_2` < 4.0 and `solemne_3` < 4.0 and `solemne_4` < 4.0 ")
# estudiantes_aprovados = ESTUDIANTES[~ESTUDIANTES['id_estudiante'].isin(estudiantes_reprobados['id_estudiante'])]
df_users = ESTUDIANTES.copy()
df_users.head()

2024-12-02 00:34:58.120856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733110498.383566    6759 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733110498.431380    6759 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 00:34:58.997332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,id_estudiante,programa,exitosos,fallidos,solemne_1,solemne_2,solemne_3,solemne_4,score_a,score_p,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,0,INGENIERIA INDUSTRIAL,15,24,6.2,6.8,5.1,6.0,5.0,4.0,...,0,1,0,0,0,0,0,0,0,0
1,1,INGENIERIA CIVIL INDUSTRIAL,10,6,7.0,6.9,7.0,7.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0
2,2,INGENIERIA CIVIL INDUSTRIAL,11,20,5.6,6.0,4.5,5.4,5.0,4.0,...,0,0,0,0,0,0,0,0,0,0
3,3,INGENIERIA CIVIL INDUSTRIAL,7,17,3.9,4.7,3.7,4.1,3.0,3.0,...,0,0,0,0,0,0,0,0,0,0
4,4,INGENIERIA EN COMPUTACION E INFORMATICA,13,18,4.6,6.6,4.7,5.3,6.0,4.0,...,0,1,0,0,0,0,0,0,0,0


In [2]:
# División inicial en train y test
train_data, test_data = train_test_split(df_users, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
# División adicional en train y validation
train_data, validation_data = train_test_split(train_data, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)

# Crear escalador y codificador
scaler = MinMaxScaler()
label_encoder = LabelEncoder()

# Columnas a normalizar y codificar
columns_to_normalize = ['exitosos', 'fallidos', 'solemne_1', 'solemne_2', 'solemne_3', 'solemne_4', 'score_a', 'score_p', 'score_d', 'score_s']
column_to_encode = 'programa'

# Ajustar en el conjunto de entrenamiento
scaler.fit(train_data[columns_to_normalize])        # Ajustar el escalador
label_encoder.fit(train_data[column_to_encode])     # Ajustar el codificador

# Entrenamiento
train_data[columns_to_normalize] = scaler.transform(train_data[columns_to_normalize])
train_data[column_to_encode] = label_encoder.transform(train_data[column_to_encode])

# Validación
validation_data[columns_to_normalize] = scaler.transform(validation_data[columns_to_normalize])
validation_data[column_to_encode] = label_encoder.transform(validation_data[column_to_encode])

# Prueba
test_data[columns_to_normalize] = scaler.transform(test_data[columns_to_normalize])
test_data[column_to_encode] = label_encoder.transform(test_data[column_to_encode])

features_users_data = ['id_estudiante', 'programa', 'exitosos', 'fallidos', 'score_a', 'score_p', 'score_d', 'score_s']
features_users_inte = ['id_estudiante'] + [f"e{i}" for i in range(len(df_items))]

df_test_users = test_data[features_users_data]
df_train_users = train_data[features_users_data]
df_validation_users = validation_data[features_users_data]

df_test_interacciones = test_data[features_users_inte]
df_train_interacciones = train_data[features_users_inte]
df_validation_interacciones = validation_data[features_users_inte]


# # Filtrar estudiantes aprobados según las condiciones
estudiantes_aprovados = ESTUDIANTES[~ESTUDIANTES['id_estudiante'].isin(
    ESTUDIANTES.query("`solemne_1` < 4.0 and `solemne_2` < 4.0 and `solemne_3` < 4.0 and `solemne_4` < 4.0")['id_estudiante']
)]

# Total de ítems
n = len(EJERCICIOS)

# Últimas 'n' columnas
columnas = ESTUDIANTES.columns[-n:]

# Limpiar nombres de columnas eliminando la 'e' al inicio
columnas_limpias = columnas.str.replace('e', '').astype(int)

# Renombrar las columnas temporalmente para evitar problemas
estudiantes_aprovados.columns = list(ESTUDIANTES.columns[:-n]) + columnas_limpias.tolist()

# Calcular la suma para cada ítem y convertir a diccionario
popularidad_items = estudiantes_aprovados.iloc[:, -n:].sum(axis=0).to_dict()


In [3]:
popularidad_items

{0: 603,
 1: 275,
 2: 17,
 3: 542,
 4: 733,
 5: 0,
 6: 277,
 7: 195,
 8: 21,
 9: 42,
 10: 599,
 11: 38,
 12: 69,
 13: 233,
 14: 0,
 15: 0,
 16: 0,
 17: 737,
 18: 389,
 19: 0,
 20: 0,
 21: 37,
 22: 650,
 23: 590,
 24: 0,
 25: 730,
 26: 678,
 27: 0,
 28: 168,
 29: 653,
 30: 0,
 31: 4,
 32: 395,
 33: 11,
 34: 0,
 35: 347,
 36: 0,
 37: 184,
 38: 87,
 39: 0,
 40: 0,
 41: 0,
 42: 472,
 43: 0,
 44: 314,
 45: 0,
 46: 0,
 47: 0,
 48: 4,
 49: 0,
 50: 0,
 51: 0,
 52: 0}

# CLASES DE DOS TORRES

In [4]:
class UserTower(nn.Module):
    """
    Red neuronal para procesar características de los usuarios.
    """
    def __init__(self, user_input_size, embedding_size, dropout_rate):
        super(UserTower, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(user_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        # Generar embedding para las características del usuario
        return self.fc(x)


class ItemTower(nn.Module):
    """
    Red neuronal para procesar características de los ítems.
    """
    def __init__(self, item_input_size, embedding_size, dropout_rate):
        super(ItemTower, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(item_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        # Generar embedding para las características del ítem
        return self.fc(x)


class TwoTowerModel(nn.Module):
    """
    Modelo de dos torres para calcular la afinidad entre usuarios e ítems.
    """
    def __init__(self, user_input_size, item_input_size, embedding_size, dropout_rate):
        super(TwoTowerModel, self).__init__()
        self.user_tower = UserTower(user_input_size, embedding_size, dropout_rate)
        self.item_tower = ItemTower(item_input_size, embedding_size, dropout_rate)

    def forward(self, user_input, item_input):
        # Generar embeddings del usuario y del ítem
        user_embedding = self.user_tower(user_input)
        item_embedding = self.item_tower(item_input)
        
        # Calcular el score como el producto punto entre los embeddings
        score = torch.sum(user_embedding * item_embedding, dim=1)
        return torch.sigmoid(score)


In [4]:
class UserTowerV1(nn.Module):
    """
    Red neuronal para procesar características de los usuarios con más capas.
    """
    def __init__(self, user_input_size, embedding_size, dropout_rate):
        super(UserTowerV1, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(user_input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        return self.fc(x)


class ItemTowerV1(nn.Module):
    """
    Red neuronal para procesar características de los ítems con más capas.
    """
    def __init__(self, item_input_size, embedding_size, dropout_rate):
        super(ItemTowerV1, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(item_input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        return self.fc(x)


class TwoTowerModelV1(nn.Module):
    """
    Modelo de dos torres para calcular la afinidad entre usuarios e ítems.
    """
    def __init__(self, user_input_size, item_input_size, embedding_size, dropout_rate):
        super(TwoTowerModelV1, self).__init__()
        self.user_tower = UserTowerV1(user_input_size, embedding_size, dropout_rate)
        self.item_tower = ItemTowerV1(item_input_size, embedding_size, dropout_rate)

    def forward(self, user_input, item_input):
        user_embedding = self.user_tower(user_input)
        item_embedding = self.item_tower(item_input)
        score = torch.sum(user_embedding * item_embedding, dim=1)
        return torch.sigmoid(score)



In [5]:
class UserTowerV2(nn.Module):
    """
    Red neuronal para procesar características de los usuarios sin Batch Normalization y con LeakyReLU.
    """
    def __init__(self, user_input_size, embedding_size, dropout_rate):
        super(UserTowerV2, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(user_input_size, 256),  # Aumentar la dimensión inicial
            nn.LeakyReLU(negative_slope=0.1),  # Activación LeakyReLU
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),  # Segunda capa con menor dimensión
            nn.LeakyReLU(negative_slope=0.1),  # Activación LeakyReLU
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)  # Proyectar al tamaño del embedding
        )

    def forward(self, x):
        return self.fc(x)


class ItemTowerV2(nn.Module):
    """
    Red neuronal para procesar características de los ítems sin Batch Normalization y con LeakyReLU.
    """
    def __init__(self, item_input_size, embedding_size, dropout_rate):
        super(ItemTowerV2, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(item_input_size, 256),  # Aumentar la dimensión inicial
            nn.LeakyReLU(negative_slope=0.1),  # Activación LeakyReLU
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),  # Segunda capa con menor dimensión
            nn.LeakyReLU(negative_slope=0.1),  # Activación LeakyReLU
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)  # Proyectar al tamaño del embedding
        )

    def forward(self, x):
        return self.fc(x)


class TwoTowerModelV2(nn.Module):
    """
    Modelo de dos torres actualizado sin Batch Normalization y con LeakyReLU.
    """
    def __init__(self, user_input_size, item_input_size, embedding_size, dropout_rate):
        super(TwoTowerModelV2, self).__init__()
        self.user_tower = UserTowerV2(user_input_size, embedding_size, dropout_rate)
        self.item_tower = ItemTowerV2(item_input_size, embedding_size, dropout_rate)

    def forward(self, user_input, item_input):
        # Embeddings del usuario y del ítem
        user_embedding = self.user_tower(user_input)
        item_embedding = self.item_tower(item_input)

        # Afinidad mediante producto punto
        score = torch.sum(user_embedding * item_embedding, dim=1)
        return torch.sigmoid(score)


In [6]:
class UserTowerV3(nn.Module):
    """
    Red neuronal para procesar características de los usuarios.
    """
    def __init__(self, user_input_size, embedding_size, dropout_rate):
        super(UserTowerV3, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(user_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        return self.fc(x)


class ItemTowerV3(nn.Module):
    """
    Red neuronal para procesar características de los ítems.
    """
    def __init__(self, item_input_size, embedding_size, dropout_rate):
        super(ItemTowerV3, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(item_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        return self.fc(x)


class TwoTowerModelV3(nn.Module):
    """
    Modelo de dos torres con afinidad basada en la distancia coseno.
    """
    def __init__(self, user_input_size, item_input_size, embedding_size, dropout_rate):
        super(TwoTowerModelV3, self).__init__()
        self.user_tower = UserTowerV3(user_input_size, embedding_size, dropout_rate)
        self.item_tower = ItemTowerV3(item_input_size, embedding_size, dropout_rate)
        self.cosine_similarity = torch.nn.CosineSimilarity(dim=1)

    def forward(self, user_input, item_input):
        user_embedding = self.user_tower(user_input)
        item_embedding = self.item_tower(item_input)
        score = self.cosine_similarity(user_embedding, item_embedding)
        return torch.sigmoid(score)


In [7]:
class UserTowerV4(nn.Module):
    """
    Red neuronal para procesar características de los usuarios.
    """
    def __init__(self, user_input_size, embedding_size, dropout_rate):
        super(UserTowerV4, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(user_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        return self.fc(x)


class ItemTowerV4(nn.Module):
    """
    Red neuronal para procesar características de los ítems.
    """
    def __init__(self, item_input_size, embedding_size, dropout_rate):
        super(ItemTowerV4, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(item_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, embedding_size)
        )

    def forward(self, x):
        return self.fc(x)


class TwoTowerModelV4(nn.Module):
    """
    Modelo de dos torres con embeddings concatenados y red adicional.
    """
    def __init__(self, user_input_size, item_input_size, embedding_size, dropout_rate):
        super(TwoTowerModelV4, self).__init__()
        self.user_tower = UserTowerV4(user_input_size, embedding_size, dropout_rate)
        self.item_tower = ItemTowerV4(item_input_size, embedding_size, dropout_rate)
        self.fc = nn.Sequential(
            nn.Linear(embedding_size * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1)
        )

    def forward(self, user_input, item_input):
        user_embedding = self.user_tower(user_input)
        item_embedding = self.item_tower(item_input)
        combined_embedding = torch.cat((user_embedding, item_embedding), dim=1)
        score = self.fc(combined_embedding).squeeze()
        return torch.sigmoid(score)


# SISTEMA DE RECOMENDACION

In [5]:
class TwoTowerRecommenderSystem:
    """
    Sistema de recomendación basado en el modelo de dos torres.
    """
    def __init__(self, two_tower_model, user_input_size: int, item_input_size: int, embedding_size: int = 64, dropout_rate: float = 0.5, learning_rate: float = 0.001, optimizer_system=optim.Adam, criterion_system=nn.BCELoss):
        
        # Inicializar el modelo, optimizador y función de pérdida
        self.model = two_tower_model(user_input_size, item_input_size, embedding_size, dropout_rate)
        self.optimizer = optimizer_system(self.model.parameters(), lr=learning_rate, weight_decay=1e-4)
        self.criterion = criterion_system()

    def save_model(self, file_path: str):
        # Guardar el estado del modelo
        torch.save(self.model.state_dict(), file_path)
        print(f"[+] Modelo guardado en {file_path}")

    def load_model(self, file_path: str):
        # Cargar el estado del modelo
        self.model.load_state_dict(torch.load(file_path))
        self.model.eval()
        print(f"[+] Modelo cargado desde {file_path}")

    def load_items(self, df_items: DataFrame):
        """
        Cargar ítems desde un DataFrame y convertirlos en tensores.
        - `df_items`: DataFrame de items con características (con ID).
        """
        self.item_inputs = torch.tensor(df_items.iloc[:, 1:].values).float()
        print(f"[+] Ítems cargados: {self.item_inputs.size(0)} ítems con {self.item_inputs.size(1)} características cada uno.")

    def train(self, df_users: DataFrame, df_interactions: DataFrame, epochs: int = 30):
        """
        Entrenar el modelo utilizando datos de usuarios e interacciones.
        - `df_users`: DataFrame de usuarios con características (con ID).
        - `df_interactions`: DataFrame de interacciones binarias usuario-ítem (con ID).
        """

        if self.item_inputs is None:
            raise ValueError("[-] Datos de los items no fueron cargados. Usa load_items() primero.")

        # Convertir datos de usuarios e interacciones a tensores
        user_inputs = torch.tensor(df_users.iloc[:, 1:].values).float()
        interactions = torch.tensor(df_interactions.iloc[:, 1:].values).float()

        # Dimensiones de entrada
        num_users = user_inputs.size(0) # Número de usuarios (n)
        num_items = self.item_inputs.size(0) # Número de ítems (k)

        # Expandir datos de usuario e ítem para todas las combinaciones usuario-ítem
        user_input_expanded = user_inputs.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, user_inputs.size(1)) # (n * k, m)
        item_input_expanded = self.item_inputs.repeat(num_users, 1) # (n * k, h)
        
        # Aplanar las etiquetas de interacciones para todas las combinaciones usuario-ítem
        labels = interactions.flatten() # Tensor de tamaño (n * k)
        
        # Verificar dimensiones
        assert user_input_expanded.size(0) == item_input_expanded.size(0) == labels.size(0), \
            f"[-] Dimensiones incompatibles: user_input_expanded={user_input_expanded.size(0)}, " \
            f"[-] item_input_expanded={item_input_expanded.size(0)}, labels={labels.size(0)}"

        # Proceso de entrenamiento
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            output = self.model(user_input_expanded, item_input_expanded)
            loss = self.criterion(output, labels)
            loss.backward()
            self.optimizer.step()
            print(f"[+] Epoch {epoch + 1}/{epochs} => Loss: {loss.item():.4f}")    

    def evaluate(self, df_users: DataFrame, df_interactions: DataFrame, k: int = 10):
        """
        Evalúa el modelo con un nuevo conjunto de datos de usuarios e interacciones.

        - `df_users`: DataFrame de usuarios con características (con ID).
        - `df_interactions`: DataFrame de interacciones binarias usuario-ítem (con ID).
        - `k`: Número de ítems a considerar para las métricas top-k.
        """

        # Convertir datos a tensores
        user_inputs = torch.tensor(df_users.iloc[:, 1:].values).float()
        interactions = torch.tensor(df_interactions.iloc[:, 1:].values).float()

        # Dimensiones
        num_users = user_inputs.size(0)
        num_items = self.item_inputs.size(0)

        # Expandir para todas las combinaciones usuario-ítem
        user_input_expanded = user_inputs.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, user_inputs.size(1))
        item_input_expanded = self.item_inputs.repeat(num_users, 1)

        # Etiquetas reales
        labels = interactions

        # Predicciones del modelo
        self.model.eval()
        with torch.no_grad():
            output = self.model(user_input_expanded, item_input_expanded).reshape(num_users, num_items)

        # Calcular métricas
        precisions, recalls, ndcgs = [], [], []
        for user_idx in range(num_users):
            true_labels = labels[user_idx]
            pred_scores = output[user_idx]

            precisions.append(self.precision_at_k(true_labels, pred_scores, k))
            recalls.append(self.recall_at_k(true_labels, pred_scores, k))
            ndcgs.append(self.ndcg_at_k(true_labels, pred_scores, k))

        # Promediar métricas
        mean_precision = sum(precisions) / num_users
        mean_recall = sum(recalls) / num_users
        mean_ndcg = sum(ndcgs) / num_users

        print(f"[+] Evaluation Results - Precision@{k}: {mean_precision:.4f}, Recall@{k}: {mean_recall:.4f}, NDCG@{k}: {mean_ndcg:.4f}")

    def precision_at_k(self, true_labels, pred_scores, k):
        """
        Calcula Precision@k para un usuario.
        """
        _, top_k_indices = torch.topk(pred_scores, k)   # Índices de los top-k ítems predichos
        top_k_pred = torch.zeros_like(true_labels)      # Inicializar predicciones binarias
        top_k_pred[top_k_indices] = 1                   # Marcar los top-k como predichos

        num_true_positives = torch.sum(top_k_pred * true_labels).item()  # Ítems relevantes en top-k
        precision = num_true_positives / k                               # Precisión
        return precision

    def recall_at_k(self, true_labels, pred_scores, k):
        """
        Calcula Recall@k para un usuario.
        """
        _, top_k_indices = torch.topk(pred_scores, k)   # Índices de los top-k ítems predichos
        top_k_pred = torch.zeros_like(true_labels)      # Inicializar predicciones binarias
        top_k_pred[top_k_indices] = 1                   # Marcar los top-k como predichos

        num_true_positives = torch.sum(top_k_pred * true_labels).item()  # Ítems relevantes en top-k
        num_relevant_items = torch.sum(true_labels).item()               # Ítems relevantes reales
        recall = num_true_positives / num_relevant_items if num_relevant_items > 0 else 0
        return recall

    def ndcg_at_k(self, true_labels, pred_scores, k):
        """
        Calcula NDCG@k para un usuario.
        """
        _, top_k_indices = torch.topk(pred_scores, k)                           # Índices de los top-k ítems predichos
        ideal_sorted_labels = torch.sort(true_labels, descending=True)[0][:k]   # Relevancias ideales ordenadas

        # DCG (Discounted Cumulative Gain)
        dcg = torch.sum(true_labels[top_k_indices] / torch.log2(torch.arange(2, k + 2).float())).item()

        # IDCG (Ideal Discounted Cumulative Gain)
        ideal_dcg = torch.sum(ideal_sorted_labels / torch.log2(torch.arange(2, k + 2).float())).item()

        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0
        return ndcg

    def recommend(self, user_features, interacted_items, top_k: int = 10):
        if not hasattr(self, 'item_inputs'):
            raise ValueError("[-] Los ítems no han sido cargados. Usa load_items() primero.")

        self.model.eval()
        with torch.no_grad():
            # Generar embedding del usuario
            user_embedding = self.model.user_tower(user_features.unsqueeze(0))  # (1, embedding_size)
            # Generar embeddings de los ítems
            item_embeddings = self.model.item_tower(self.item_inputs)  # (num_items, embedding_size)

            # Calcular los scores
            scores = torch.matmul(item_embeddings, user_embedding.squeeze().unsqueeze(1)).squeeze()

            # Validar `interacted_items`
            if not interacted_items:
                print("[-] Warning: La lista interacted_items está vacía.")
            else:
                for idx in interacted_items:
                    if idx < 0 or idx >= len(scores):
                        raise ValueError(f"Índice fuera de rango: {idx}")

            # Penalizar ítems ya interactuados
            for idx in interacted_items:
                scores[idx] = float('-inf')  # Penalizar ítems interactuados con -inf

            # # Validar los valores penalizados
            # penalized_scores = [scores[idx].item() for idx in interacted_items]
            # print("Valores penalizados en índices interactuados:", penalized_scores)

            # Obtener los top-k ítems
            top_k_scores, top_k_indices = torch.topk(scores, top_k)

            # Generar recomendaciones
            recommendations = [
                (idx, score)
                for idx, score in zip(top_k_indices.tolist(), top_k_scores.tolist())
            ]

            # # Validar que los ítems recomendados no estén en los interactuados
            # for idx, _ in recommendations:
            #     if idx in interacted_items:
            #         print(f"Error: Ítem interactuado {idx} fue recomendado.")

            return recommendations

    def evaluate_general_relevance(self, recommended_items, popularity):
        """
        Evalúa la relevancia general de las recomendaciones.
        - `recommended_items`: Diccionario {usuario: lista de ítems recomendados}.
        - `popularity`: Diccionario {ítem: popularidad (frecuencia de interacciones)}.
        """
        # Coverage
        unique_items = set(item for items in recommended_items.values() for item in items)
        coverage = len(unique_items) / self.item_inputs.shape[0]

        # Calcular un valor mínimo positivo para ítems sin interacciones
        min_popularity = 1 / (sum(popularity.values()) + 1)  # Evita división por cero

        # Novelty
        novelty = 0
        total_recommendations = 0
        for items in recommended_items.values():
            for item in items:
                # Obtener la popularidad del ítem, usando el mínimo si no tiene interacciones
                item_popularity = max(popularity.get(item, 0), min_popularity)
                novelty += -torch.log2(torch.tensor(item_popularity))
                total_recommendations += 1
        novelty /= total_recommendations

        # Popularity Bias
        avg_popularity = 0
        for items in recommended_items.values():
            avg_popularity += sum(popularity.get(item, 0) for item in items)
        avg_popularity /= total_recommendations

        # Imprimir métricas
        print(f"Coverage: {coverage:.4f}, Novelty: {novelty:.4f}, Popularity Bias: {avg_popularity:.4f}")
        return coverage, novelty, avg_popularity


# MODELOS

In [6]:
# Inicializar el sistema recomendador 
recommender = TwoTowerRecommenderSystem(
    two_tower_model=TwoTowerModel,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)


# recommender1 = TwoTowerRecommenderSystem(
#     two_tower_model=TwoTowerModelV1,
#     user_input_size=df_train_users.shape[1] - 1,
#     item_input_size=df_items.shape[1] - 1,
# )

# recommender2 = TwoTowerRecommenderSystem(
#     two_tower_model=TwoTowerModelV2,
#     user_input_size=df_train_users.shape[1] - 1,
#     item_input_size=df_items.shape[1] - 1,
# )


# recommender3 = TwoTowerRecommenderSystem(
#     two_tower_model=TwoTowerModelV3,
#     user_input_size=df_train_users.shape[1] - 1,
#     item_input_size=df_items.shape[1] - 1,
# )


# recommender4 = TwoTowerRecommenderSystem(
#     two_tower_model=TwoTowerModelV4,
#     user_input_size=df_train_users.shape[1] - 1,
#     item_input_size=df_items.shape[1] - 1,
# )

In [7]:
# Cargar datos en el sistema

recommender.load_items(
    df_items=df_items
)

# recommender1.load_items(
#     df_items=df_items
# )

# recommender2.load_items(
#     df_items=df_items
# )

# recommender3.load_items(
#     df_items=df_items
# )

# recommender4.load_items(
#     df_items=df_items
# )

[+] Ítems cargados: 53 ítems con 12 características cada uno.


In [8]:
# Entrenar el modelo para recomendar
recommender.train(
    df_users=df_train_users,
    df_interactions=df_train_interacciones,
    epochs=30
)

[+] Epoch 1/30 => Loss: 0.7548
[+] Epoch 2/30 => Loss: 0.6243
[+] Epoch 3/30 => Loss: 0.5453
[+] Epoch 4/30 => Loss: 0.5088
[+] Epoch 5/30 => Loss: 0.4933
[+] Epoch 6/30 => Loss: 0.4813
[+] Epoch 7/30 => Loss: 0.4727
[+] Epoch 8/30 => Loss: 0.4582
[+] Epoch 9/30 => Loss: 0.4450
[+] Epoch 10/30 => Loss: 0.4267
[+] Epoch 11/30 => Loss: 0.4093
[+] Epoch 12/30 => Loss: 0.3927
[+] Epoch 13/30 => Loss: 0.3804
[+] Epoch 14/30 => Loss: 0.3739
[+] Epoch 15/30 => Loss: 0.3710
[+] Epoch 16/30 => Loss: 0.3707
[+] Epoch 17/30 => Loss: 0.3674
[+] Epoch 18/30 => Loss: 0.3669
[+] Epoch 19/30 => Loss: 0.3586
[+] Epoch 20/30 => Loss: 0.3533
[+] Epoch 21/30 => Loss: 0.3485
[+] Epoch 22/30 => Loss: 0.3474
[+] Epoch 23/30 => Loss: 0.3423
[+] Epoch 24/30 => Loss: 0.3453
[+] Epoch 25/30 => Loss: 0.3434
[+] Epoch 26/30 => Loss: 0.3415
[+] Epoch 27/30 => Loss: 0.3402
[+] Epoch 28/30 => Loss: 0.3365
[+] Epoch 29/30 => Loss: 0.3319
[+] Epoch 30/30 => Loss: 0.3269


In [12]:
# Entrenar el modelo para recomendar
recommender1.train(
    df_users=df_train_users,
    df_interactions=df_train_interacciones,
    epochs=30
)

[+] Epoch 1/30 => Loss: 0.6597
[+] Epoch 2/30 => Loss: 0.5749
[+] Epoch 3/30 => Loss: 0.5117
[+] Epoch 4/30 => Loss: 0.4859
[+] Epoch 5/30 => Loss: 0.4847
[+] Epoch 6/30 => Loss: 0.4706
[+] Epoch 7/30 => Loss: 0.4383
[+] Epoch 8/30 => Loss: 0.4086
[+] Epoch 9/30 => Loss: 0.3873
[+] Epoch 10/30 => Loss: 0.3726
[+] Epoch 11/30 => Loss: 0.3571
[+] Epoch 12/30 => Loss: 0.3397
[+] Epoch 13/30 => Loss: 0.3250
[+] Epoch 14/30 => Loss: 0.3187
[+] Epoch 15/30 => Loss: 0.3188
[+] Epoch 16/30 => Loss: 0.3260
[+] Epoch 17/30 => Loss: 0.3261
[+] Epoch 18/30 => Loss: 0.3213
[+] Epoch 19/30 => Loss: 0.3130
[+] Epoch 20/30 => Loss: 0.3049
[+] Epoch 21/30 => Loss: 0.2973
[+] Epoch 22/30 => Loss: 0.2904
[+] Epoch 23/30 => Loss: 0.2878
[+] Epoch 24/30 => Loss: 0.2867
[+] Epoch 25/30 => Loss: 0.2874
[+] Epoch 26/30 => Loss: 0.2858
[+] Epoch 27/30 => Loss: 0.2822
[+] Epoch 28/30 => Loss: 0.2778
[+] Epoch 29/30 => Loss: 0.2757
[+] Epoch 30/30 => Loss: 0.2702


In [13]:
# Entrenar el modelo para recomendar
recommender2.train(
    df_users=df_train_users,
    df_interactions=df_train_interacciones,
    epochs=30
)

[+] Epoch 1/30 => Loss: 0.7115
[+] Epoch 2/30 => Loss: 0.5949
[+] Epoch 3/30 => Loss: 0.5245
[+] Epoch 4/30 => Loss: 0.4982
[+] Epoch 5/30 => Loss: 0.4995
[+] Epoch 6/30 => Loss: 0.4922
[+] Epoch 7/30 => Loss: 0.4655
[+] Epoch 8/30 => Loss: 0.4382
[+] Epoch 9/30 => Loss: 0.4185
[+] Epoch 10/30 => Loss: 0.4066
[+] Epoch 11/30 => Loss: 0.3939
[+] Epoch 12/30 => Loss: 0.3776
[+] Epoch 13/30 => Loss: 0.3602
[+] Epoch 14/30 => Loss: 0.3474
[+] Epoch 15/30 => Loss: 0.3441
[+] Epoch 16/30 => Loss: 0.3465
[+] Epoch 17/30 => Loss: 0.3489
[+] Epoch 18/30 => Loss: 0.3473
[+] Epoch 19/30 => Loss: 0.3417
[+] Epoch 20/30 => Loss: 0.3379
[+] Epoch 21/30 => Loss: 0.3290
[+] Epoch 22/30 => Loss: 0.3221
[+] Epoch 23/30 => Loss: 0.3144
[+] Epoch 24/30 => Loss: 0.3112
[+] Epoch 25/30 => Loss: 0.3103
[+] Epoch 26/30 => Loss: 0.3084
[+] Epoch 27/30 => Loss: 0.3082
[+] Epoch 28/30 => Loss: 0.3046
[+] Epoch 29/30 => Loss: 0.3045
[+] Epoch 30/30 => Loss: 0.2999


In [14]:
# Entrenar el modelo para recomendar
recommender3.train(
    df_users=df_train_users,
    df_interactions=df_train_interacciones,
    epochs=30
)

[+] Epoch 1/30 => Loss: 0.6910
[+] Epoch 2/30 => Loss: 0.6620
[+] Epoch 3/30 => Loss: 0.6368
[+] Epoch 4/30 => Loss: 0.6158
[+] Epoch 5/30 => Loss: 0.5989
[+] Epoch 6/30 => Loss: 0.5862
[+] Epoch 7/30 => Loss: 0.5757
[+] Epoch 8/30 => Loss: 0.5675
[+] Epoch 9/30 => Loss: 0.5616
[+] Epoch 10/30 => Loss: 0.5563
[+] Epoch 11/30 => Loss: 0.5518
[+] Epoch 12/30 => Loss: 0.5477
[+] Epoch 13/30 => Loss: 0.5440
[+] Epoch 14/30 => Loss: 0.5410
[+] Epoch 15/30 => Loss: 0.5378
[+] Epoch 16/30 => Loss: 0.5351
[+] Epoch 17/30 => Loss: 0.5326
[+] Epoch 18/30 => Loss: 0.5300
[+] Epoch 19/30 => Loss: 0.5273
[+] Epoch 20/30 => Loss: 0.5249
[+] Epoch 21/30 => Loss: 0.5224
[+] Epoch 22/30 => Loss: 0.5199
[+] Epoch 23/30 => Loss: 0.5174
[+] Epoch 24/30 => Loss: 0.5149
[+] Epoch 25/30 => Loss: 0.5122
[+] Epoch 26/30 => Loss: 0.5095
[+] Epoch 27/30 => Loss: 0.5062
[+] Epoch 28/30 => Loss: 0.5040
[+] Epoch 29/30 => Loss: 0.5012
[+] Epoch 30/30 => Loss: 0.4980


In [15]:
# Entrenar el modelo para recomendar
recommender4.train(
    df_users=df_train_users,
    df_interactions=df_train_interacciones,
    epochs=30
)

[+] Epoch 1/30 => Loss: 0.6617
[+] Epoch 2/30 => Loss: 0.6367
[+] Epoch 3/30 => Loss: 0.6125
[+] Epoch 4/30 => Loss: 0.5905
[+] Epoch 5/30 => Loss: 0.5710
[+] Epoch 6/30 => Loss: 0.5534
[+] Epoch 7/30 => Loss: 0.5387
[+] Epoch 8/30 => Loss: 0.5278
[+] Epoch 9/30 => Loss: 0.5205
[+] Epoch 10/30 => Loss: 0.5171
[+] Epoch 11/30 => Loss: 0.5151
[+] Epoch 12/30 => Loss: 0.5148
[+] Epoch 13/30 => Loss: 0.5129
[+] Epoch 14/30 => Loss: 0.5095
[+] Epoch 15/30 => Loss: 0.5030
[+] Epoch 16/30 => Loss: 0.4955
[+] Epoch 17/30 => Loss: 0.4843
[+] Epoch 18/30 => Loss: 0.4757
[+] Epoch 19/30 => Loss: 0.4634
[+] Epoch 20/30 => Loss: 0.4539
[+] Epoch 21/30 => Loss: 0.4441
[+] Epoch 22/30 => Loss: 0.4358
[+] Epoch 23/30 => Loss: 0.4265
[+] Epoch 24/30 => Loss: 0.4158
[+] Epoch 25/30 => Loss: 0.4054
[+] Epoch 26/30 => Loss: 0.3949
[+] Epoch 27/30 => Loss: 0.3857
[+] Epoch 28/30 => Loss: 0.3753
[+] Epoch 29/30 => Loss: 0.3660
[+] Epoch 30/30 => Loss: 0.3584


In [9]:
# Evaluar el modelo recomendacion
recommender.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=5
)

# Evaluar el modelo recomendacion
recommender.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=10
)

# Evaluar el modelo recomendacion
recommender.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=15
)

# Evaluar el modelo recomendacion
recommender.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=20
)

[+] Evaluation Results - Precision@5: 0.7646, Recall@5: 0.3886, NDCG@5: 0.8175
[+] Evaluation Results - Precision@10: 0.7349, Recall@10: 0.6991, NDCG@10: 0.8401
[+] Evaluation Results - Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8553
[+] Evaluation Results - Precision@20: 0.5554, Recall@20: 0.9712, NDCG@20: 0.8742


In [17]:
# Evaluar el modelo recomendacion
recommender1.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=5
)

# Evaluar el modelo recomendacion
recommender1.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=10
)

# Evaluar el modelo recomendacion
recommender1.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=15
)

# Evaluar el modelo recomendacion
recommender1.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=20
)

[+] Evaluation Results - Precision@5: 0.7623, Recall@5: 0.3837, NDCG@5: 0.7889
[+] Evaluation Results - Precision@10: 0.7360, Recall@10: 0.6996, NDCG@10: 0.8433
[+] Evaluation Results - Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8562
[+] Evaluation Results - Precision@20: 0.5557, Recall@20: 0.9714, NDCG@20: 0.8924


In [18]:
# Evaluar el modelo recomendacion
recommender2.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=5
)

# Evaluar el modelo recomendacion
recommender2.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=10
)

# Evaluar el modelo recomendacion
recommender2.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=15
)

# Evaluar el modelo recomendacion
recommender2.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=20
)

[+] Evaluation Results - Precision@5: 0.7474, Recall@5: 0.3791, NDCG@5: 0.7746
[+] Evaluation Results - Precision@10: 0.7371, Recall@10: 0.7004, NDCG@10: 0.8433
[+] Evaluation Results - Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8559
[+] Evaluation Results - Precision@20: 0.5554, Recall@20: 0.9712, NDCG@20: 0.8911


In [19]:
# Evaluar el modelo recomendacion
recommender3.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=5
)

# Evaluar el modelo recomendacion
recommender3.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=10
)

# Evaluar el modelo recomendacion
recommender3.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=15
)

# Evaluar el modelo recomendacion
recommender3.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=20
)

[+] Evaluation Results - Precision@5: 0.7394, Recall@5: 0.3650, NDCG@5: 0.7562
[+] Evaluation Results - Precision@10: 0.6651, Recall@10: 0.6065, NDCG@10: 0.7834
[+] Evaluation Results - Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8073
[+] Evaluation Results - Precision@20: 0.5500, Recall@20: 0.9643, NDCG@20: 0.8566


In [20]:
# Evaluar el modelo recomendacion
recommender4.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=5
)

# Evaluar el modelo recomendacion
recommender4.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=10
)

# Evaluar el modelo recomendacion
recommender4.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=15
)

# Evaluar el modelo recomendacion
recommender4.evaluate(
    df_users=df_validation_users,
    df_interactions=df_validation_interacciones,
    k=20
)

[+] Evaluation Results - Precision@5: 0.6640, Recall@5: 0.3229, NDCG@5: 0.7159
[+] Evaluation Results - Precision@10: 0.6457, Recall@10: 0.5824, NDCG@10: 0.7637
[+] Evaluation Results - Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.7992
[+] Evaluation Results - Precision@20: 0.5403, Recall@20: 0.9519, NDCG@20: 0.8422


In [10]:
# Diccionario para almacenar las recomendaciones
recommended_items = {}
# recommended_items1 = {}
# recommended_items2 = {}
# recommended_items3 = {}
# recommended_items4 = {}

# Iterar sobre todos los usuarios en el conjunto de prueba
for user_id in df_test_users['id_estudiante']:
    # Obtener las características del usuario actual
    user_features = torch.tensor(df_test_users[df_test_users['id_estudiante'] == user_id].iloc[:, 1:].values).float()

    # Obtener los ítems ya interactuados por el usuario actual
    interacted_items = df_test_interacciones[df_test_interacciones['id_estudiante'] == user_id].iloc[:, 1:].values.flatten()
    interacted_indices = [i for i, interaction in enumerate(interacted_items) if interaction == 1]

    # Generar recomendaciones para el usuario actual
    recommendations = recommender.recommend(user_features, interacted_indices, top_k=15)
    # recommendations1 = recommender1.recommend(user_features, interacted_indices, top_k=15)
    # recommendations2 = recommender2.recommend(user_features, interacted_indices, top_k=15)
    # recommendations3 = recommender3.recommend(user_features, interacted_indices, top_k=15)
    # recommendations4 = recommender4.recommend(user_features, interacted_indices, top_k=15)

    # Almacenar solo los índices de los ítems recomendados (sin los scores)
    recommended_items[user_id] = [rec[0] for rec in recommendations]
    # recommended_items1[user_id] = [rec[0] for rec in recommendations1]
    # recommended_items2[user_id] = [rec[0] for rec in recommendations2]
    # recommended_items3[user_id] = [rec[0] for rec in recommendations3]
    # recommended_items4[user_id] = [rec[0] for rec in recommendations4]

In [12]:
recommended_items

{564: [7, 3, 42, 29, 35, 0, 1, 44, 6, 50, 52, 16, 27, 15, 19],
 155: [7, 23, 18, 3, 32, 42, 13, 35, 0, 1, 50, 6, 44, 27, 52],
 98: [7, 32, 13, 1, 50, 44, 6, 27, 52, 16, 19, 14, 15, 21, 28],
 1023: [29, 26, 0, 1, 50, 6, 44, 52, 16, 27, 19, 14, 15, 38, 28],
 901: [7, 3, 42, 35, 1, 50, 44, 6, 52, 16, 27, 19, 14, 15, 9],
 988: [3, 42, 13, 1, 44, 6, 50, 52, 16, 27, 15, 14, 19, 38, 31],
 319: [25, 7, 42, 13, 50, 27, 52, 16, 19, 14, 15, 28, 21, 11, 31],
 507: [7, 23, 18, 3, 29, 42, 32, 13, 35, 0, 1, 50, 44, 6, 16],
 1300: [7, 42, 29, 10, 1, 50, 44, 6, 52, 16, 27, 19, 14, 15, 37],
 1001: [7, 22, 23, 42, 3, 32, 29, 10, 18, 26, 35, 13, 0, 1, 6],
 1110: [1, 50, 44, 6, 52, 16, 27, 19, 14, 15, 21, 37, 31, 38, 11],
 979: [25, 4, 7, 18, 10, 32, 13, 35, 0, 1, 44, 6, 50, 52, 16],
 1013: [1, 50, 44, 6, 52, 16, 27, 19, 14, 15, 21, 37, 31, 38, 11],
 737: [4, 7, 23, 22, 18, 10, 29, 32, 3, 42, 26, 35, 13, 0, 1],
 695: [4, 7, 42, 3, 32, 29, 18, 26, 35, 13, 0, 1, 50, 6, 44],
 1189: [7, 42, 10, 13, 1, 50, 44, 

In [11]:
coverage, novelty, popularity_bias = recommender.evaluate_general_relevance(
    recommended_items=recommended_items,
    popularity=popularidad_items
)

Coverage: 0.8113, Novelty: -0.2959, Popularity Bias: 212.2079


In [23]:
coverage1, novelty1, popularity_bias1 = recommender1.evaluate_general_relevance(
    recommended_items=recommended_items1,
    popularity=popularidad_items
)

Coverage: 0.8113, Novelty: -4.8450, Popularity Bias: 227.2877


In [24]:
coverage2, novelty2, popularity_bias2 = recommender2.evaluate_general_relevance(
    recommended_items=recommended_items2,
    popularity=popularidad_items
)

Coverage: 0.8113, Novelty: -3.9376, Popularity Bias: 221.6374


In [25]:
coverage3, novelty3, popularity_bias3 = recommender3.evaluate_general_relevance(
    recommended_items=recommended_items3,
    popularity=popularidad_items
)

Coverage: 0.7547, Novelty: 1.7276, Popularity Bias: 200.2956


In [26]:
coverage4, novelty4, popularity_bias4 = recommender4.evaluate_general_relevance(
    recommended_items=recommended_items1,
    popularity=popularidad_items
)

Coverage: 0.8113, Novelty: -4.8450, Popularity Bias: 227.2877


In [27]:
# Supongamos que el usuario con ID=123 tiene las siguientes características:
user_id = 1023
user_features = torch.tensor(df_test_users[df_test_users['id_estudiante'] == user_id].iloc[:, 1:].values).float()

# Ítems ya interactuados por el usuario
interacted_items = df_test_interacciones[df_test_interacciones['id_estudiante'] == user_id].iloc[:, 1:].values.flatten()
interacted_indices = [i for i, interaction in enumerate(interacted_items) if interaction == 1]
print(interacted_indices)

# Generar recomendaciones
reomendacions = recommender.recommend(user_features, interacted_indices, top_k=15)

print(reomendacions)
print([x[0] for x in reomendacions])

[3, 4, 7, 10, 13, 17, 18, 22, 23, 25, 32, 35, 42]
[(26, 0.3007104992866516), (29, 0.30071043968200684), (0, -0.9705798625946045), (1, -1.441230058670044), (6, -1.7118687629699707), (44, -1.7118687629699707), (50, -2.1610219478607178), (16, -3.2638938426971436), (52, -3.2638940811157227), (27, -3.335946559906006), (28, -3.6505961418151855), (38, -3.6505961418151855), (21, -3.6505961418151855), (31, -3.6505961418151855), (37, -3.6505961418151855)]
[26, 29, 0, 1, 6, 44, 50, 16, 52, 27, 28, 38, 21, 31, 37]
